In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd
import time
import json

In [2]:
# open broswer for next output
execute_path = {"executable_path": "/usr/local/bin/chromedriver"}
browser = Browser("chrome", **execute_path, headless = False)

In [3]:
# get the table in url_new and add the linker/url for all data
url_news = 'https://www.oeis.ucf.edu/buildings'
browser.visit(url_news)
html = browser.html
soup = bs(html, 'html.parser')
time.sleep(1)
first_bs = soup.find('table', {'id': 'buildings'})
df_list = []
for row in first_bs.find_all('tr'):
    for td in row.find_all('a'):
        df_list.append({'html': td['href'], 'Name': td.text[1:-1]})
df_a = pd.DataFrame(df_list)
df_table = pd.read_html(url_news)
df_real = pd.merge(df_a, df_table[0], on = 'Name', how = 'inner')
df_real.head()

,html,Name,Number,Type,Area (sqft),EUI (kBTU/sqft),LEED
0,https://www.oeis.ucf.edu/buildings/5,Alpha Delta Pi,406,Residence Hall,5477,270.56,NaN
1,https://www.oeis.ucf.edu/buildings/65,Alpha Tau Omega,410,Residence Hall,10000,61.70,NaN
2,https://www.oeis.ucf.edu/buildings/60,Alpha Xi Delta,404,Residence Hall,5200,534.42,NaN
3,https://www.oeis.ucf.edu/buildings/45,AMPAC Research Facility,152,Research,7432,276.40,NaN
4,https://www.oeis.ucf.edu/buildings/133,Ara Dr. Research Facility,117,Research,2720,375.14,NaN


In [17]:
# get the information of building id and square_feet to search for data
df_building = pd.read_csv('../Resources/building_metadata.csv')
sqf_list = list(df_building.loc[df_building['site_id'] == 0,'square_feet'])
df_building = df_building.loc[df_building['site_id'] == 0,:].set_index('square_feet')
df_reset = df_real.set_index('Area (sqft)')
json_list = []
df_electric = pd.DataFrame([], columns = ['timestamp', 'reading', 'building_id', 'meter'])
df_water = pd.DataFrame([], columns = ['timestamp', 'reading', 'building_id', 'meter'])
pop_index = 0

In [18]:
# test how many information in the list
real_list = list(df_real['Area (sqft)'])
len(sqf_list)

105

In [29]:
# if the loop is crash, delete the run data in loop to rerun for saving time.
temp_list = sqf_list[pop_index: ]
# main loop to scrapping
for test in temp_list:
    print(pop_index)
    # open browser
    execute_path = {"executable_path": "/usr/local/bin/chromedriver"}
    browser = Browser("chrome", **execute_path, headless = False)
    url_new = df_reset.loc[test,'html']
    # test whether there is only one building having the area information
    if_break = False
    if type(url_new) != str:
        if_break = True
        # if not check the builting year
        for i in url_new:
            browser.visit(i)
            time.sleep(1)
            html2 = browser.html
            soup2 = bs(html2, 'html.parser')
            txt = soup2.find_all('div', {'class': "col-sm-7"})[0].find_all('div')[1].text
            txt = txt[-4:]
            if txt == str(df_building.loc[test, 'year_built']):
                url_new = i
                if_break = False
                break
    if if_break:
        continue
    browser.visit(url_new)
    time.sleep(1)
    html1 = browser.html
    soup1 = bs(html1, 'html.parser')
    # print to check whether information is correct
    print('sqft: ' + str(test) + '; years: ' + str(df_building.loc[test, 'year_built']))
    print(soup1.find_all('div', {'class': "col-sm-7"})[0].find_all('div')[1].text)
    print('-'*35)
    # fill the data and click get data to get the information. (click may not work)
    browser.fill('start-date', '01/01/2017')
    browser.fill('end-date', '01/01/2019')
    browser.find_by_id('filetype').first.select('json')
    try:
        browser.find_by_value('Get Data').click()
    except:
        print('there is something wrong in ' + str(test))
        break
    # sleep enough time to avoid not loading enough
    html = browser.html
    time.sleep(3)
    soup = bs(html, 'html.parser')
    browser.quit()
    # load json
    try:
        json_real = json.loads(soup.text)
    except:
        # if there is no information, just continue
        print('there is no data in ' + str(test))
        temp_list = sqf_list[pop_index: ]
        continue
    # try to make the dataframe
    try:
        df_temp_electric = pd.DataFrame(json_real[0]['values'])
        df_temp_electric['building_id'] = df_building.loc[test, 'building_id']
        df_temp_electric['meter'] = 0
        df_electric = pd.concat([df_electric, df_temp_electric], ignore_index=True, sort=False)
    except:
        print('there is huge problem in ' + str(test))
        break
    try:
        df_temp_water = pd.DataFrame(json_real[1]['values'])
    except:
        print('there is no water data in ' + str(test))
    try:
        df_temp_water['building_id'] = df_building.loc[test, 'building_id']
        df_temp_water['meter'] = 5
        df_water = pd.concat([df_water, df_temp_water], ignore_index=True, sort=False)
    except:
        print('there is huge problem of water data in ' + str(test))
    print('*'*35)
    print(str(df_electric.shape))
    print(str(df_water.shape))
    print('*'*35)
    pop_index += 1

In [45]:
electric_list = list(df_electric['building_id'].unique())
electric_area_list = []
for i in electric_list:
    electric_area_list.append(df_building.index[df_building['building_id'] == i].values[0])
rerun_list = []
for i in sqf_list:
    if i not in electric_area_list:
        rerun_list.append(i)
pop_index = 0

In [54]:
temp_list = rerun_list[pop_index: ]
# main loop to scrapping
for test in rerun_list:
    print(pop_index)
    execute_path = {"executable_path": "/usr/local/bin/chromedriver"}
    browser = Browser("chrome", **execute_path, headless = False)
    url_new = df_reset.loc[test,'html']
    if_break = False
    if type(url_new) != str:
        if_break = True
        for i in url_new:
            browser.visit(i)
            time.sleep(1)
            html2 = browser.html
            soup2 = bs(html2, 'html.parser')
            txt = soup2.find_all('div', {'class': "col-sm-7"})[0].find_all('div')[1].text
            txt = txt[-4:]
            if txt == str(df_building.loc[test, 'year_built']):
                url_new = i
                if_break = False
                break
    if if_break:
        pop_index += 1
        print('there is no data in ' + str(test))
        continue
    browser.visit(url_new)
    time.sleep(1)
    html1 = browser.html
    soup1 = bs(html1, 'html.parser')
    print('sqft: ' + str(test) + '; years: ' + str(df_building.loc[test, 'year_built']))
    print(soup1.find_all('div', {'class': "col-sm-7"})[0].find_all('div')[1].text)
    print('-'*35)
    browser.fill('start-date', '01/01/2017')
    browser.fill('end-date', '01/01/2019')
    browser.find_by_id('filetype').first.select('json')
    try:
        browser.find_by_value('Get Data').click()
    except:
        print('there is something wrong in ' + str(test))
        break
    html = browser.html
    time.sleep(3)
    soup = bs(html, 'html.parser')
    browser.quit()
    try:
        json_real = json.loads(soup.text)
    except:
        print('there is no data in ' + str(test))
        temp_list = sqf_list[pop_index: ]
        continue
    try:
        df_temp_electric = pd.DataFrame(json_real[0]['values'])
        df_temp_electric['building_id'] = df_building.loc[test, 'building_id']
        df_temp_electric['meter'] = 0
        df_electric = pd.concat([df_electric, df_temp_electric], ignore_index=True, sort=False)
    except:
        print('there is huge problem in ' + str(test))
        break
    try:
        df_temp_water = pd.DataFrame(json_real[1]['values'])
    except:
        print('there is no water data in ' + str(test))
    try:
        df_temp_water['building_id'] = df_building.loc[test, 'building_id']
        df_temp_water['meter'] = 5
        df_water = pd.concat([df_water, df_temp_water], ignore_index=True, sort=False)
    except:
        print('there is huge problem of water data in ' + str(test))
    print('*'*35)
    print(str(df_electric.shape))
    print(str(df_water.shape))
    print('*'*35)
    pop_index += 1

1
there is no data in 59200
2
there is no data in 38686
3
there is no data in 42731
4
there is no data in 41617
5
there is no data in 53130
6
there is no data in 53130
7
sqft: 59071; years: 1980.0
59071 sqft Retail building, constructed in 1980
-----------------------------------
there is no data in 59071
7
there is no data in 24456
8
there is no data in 387638
9
there is no data in 38686
10
there is no data in 42731
11
there is no data in 41617
12
there is no data in 53130
13
there is no data in 53130
14
there is no data in 37241
15
there is no data in 24456
16
there is no data in 387638
17
there is no data in 387638
18
there is no data in 387638
19
there is no data in 387638
20
there is no data in 387638
21
there is no data in 387638
22
there is no data in 24456
23
there is no data in 37241
24
there is no data in 59200
25
sqft: 10779; years: 1970.0
10779 sqft Office building, constructed in 1970
-----------------------------------
there is no data in 10779
25
there is no data in 2009

In [55]:
df_electric.to_csv('../Large_output/csv/electric_site0.csv')
df_water.to_csv('./Large_output/csv/water_site0.csv')